# Test on different LLM model

### Evaluator LLM gpt-3.5-turbo/ RAG LLM gpt-3.5-turbo

In [1]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.integrations.llama_index import evaluate

from dotenv import load_dotenv

load_dotenv()
# Get the API key from the environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# For different LLM models
RAG_LLM = "gpt-3.5-turbo"
GENERATOR_LLM = "gpt-3.5-turbo"
CRITIC_LLM = "gpt-3.5-turbo"
EVALUATOR_LLM = "gpt-3.5-turbo"

# For different temperatures
TEMPERATURE = 0.1

# For different embeddings
RAG_EMBEDDING = "text-embedding-ada-002"
EVALUATOR_EMBEDDING = "text-embedding-ada-002"

# For different chunk
CHUNK_SIZE = 512
CHUNK_OVERLAP = 20


# Set the LLM, Embedding, and Text Splitter
Settings.llm = OpenAI(model=RAG_LLM, temperature=TEMPERATURE)
Settings.embed_model = OpenAIEmbedding(model=RAG_EMBEDDING)
Settings.text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)


# Load the documents
documents = SimpleDirectoryReader("data/Trump/").load_data()
# Create the vector index
vector_index = VectorStoreIndex.from_documents(documents)
# Create the query engine
query_engine = vector_index.as_query_engine()

# For the evaluation
# Set the LLMs
generator_llm = OpenAI(model=GENERATOR_LLM)
critic_llm = OpenAI(model=CRITIC_LLM)
evaluator_llm = OpenAI(model=EVALUATOR_LLM)
# Set the embeddings
embeddings = OpenAIEmbedding(model=EVALUATOR_EMBEDDING)

# Create the testset generator
generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

# Generate the testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    test_size=10,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

# Define the evaluation metrics
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

# convert to HF dataset
ds = testset.to_dataset()

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds,
    llm=OpenAI(model=EVALUATOR_LLM),
    embeddings=OpenAIEmbedding(model=EVALUATOR_EMBEDDING),
)

# Convert the result to pandas
result_df = result.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Running Query Engine:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs


In [2]:
result_df

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,What qualities and actions exemplified Corey C...,[Josh Shapiro said at a news conference Sunday...,Corey Comperatore's remarkable character as a ...,The answer to given question is not present in...,0.500000,0.925503,0.0,0.0
1,Who was identified as the shooter in the assas...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,Thomas Matthew Crooks was identified as the sh...,"Thomas Matthew Crooks, a 20-year-old from Beth...",1.000000,1.000000,1.0,1.0
2,What was Crooks' political affiliation based o...,[camo/huntingoutfitsinclass.HesaidCrookswouldo...,Crooks was registered as a Republican voter in...,Crooks was registered as a Republican voter in...,1.000000,0.000000,1.0,1.0
3,Who is Thomas Matthew Crooks and what was his ...,[Former President Donald Trump was shot and in...,Thomas Matthew Crooks was a 20-year-old man fr...,Thomas Matthew Crooks was the shooter who atte...,1.000000,0.000000,1.0,1.0
4,How did the Secret Service respond to the shoo...,"[Convention in Milwaukee, with his right ear b...",The Secret Service responded to the shooting i...,The Secret Service responded to the shooting i...,0.333333,0.803050,1.0,1.0
5,Who was the shooter in the attempt on former P...,[Former President Donald Trump was shot and in...,The shooter in the attempt on former President...,The answer to given question is not present in...,1.000000,0.915510,0.0,0.0
6,How did President Biden address the Pennsylvan...,[has been getting briefed by top officials and...,President Biden addressed the Pennsylvania sho...,President Biden addressed the Pennsylvania sho...,0.500000,0.946774,1.0,1.0
7,Who spotted the suspicious person at the rally...,[Who identified the suspicious person outside ...,A bystander spotted the suspicious person at t...,The answer to given question is not present in...,0.750000,0.854558,0.0,1.0
8,What did President Biden do after the Pennsylv...,"[""Hiscommitmenttohiswifeandtwodaughters,andhis...",President Biden made brief remarks at the Whit...,President Biden first responded to the shootin...,1.000000,0.900340,1.0,1.0
9,Who was responsible for the assassination atte...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,Thomas Matthew Crooks was responsible for the ...,The FBI identified the shooter as 20-year-old ...,1.000000,0.999998,1.0,1.0


### Evaluator LLM gpt-3.5-turbo/ RAG LLM gpt-4o

In [7]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.integrations.llama_index import evaluate

from dotenv import load_dotenv

load_dotenv()
# Get the API key from the environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# For different LLM models
RAG_LLM = "gpt-4o"
GENERATOR_LLM = "gpt-3.5-turbo"
CRITIC_LLM = "gpt-3.5-turbo"
EVALUATOR_LLM = "gpt-3.5-turbo"

# For different temperatures
TEMPERATURE = 0.1

# For different embeddings
RAG_EMBEDDING = "text-embedding-ada-002"
EVALUATOR_EMBEDDING = "text-embedding-ada-002"

# For different chunk
CHUNK_SIZE = 512
CHUNK_OVERLAP = 20


# Set the LLM, Embedding, and Text Splitter
Settings.llm = OpenAI(model=RAG_LLM, temperature=TEMPERATURE)
Settings.embed_model = OpenAIEmbedding(model=RAG_EMBEDDING)
Settings.text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)


# Load the documents
documents = SimpleDirectoryReader("data/Trump/").load_data()
# Create the vector index
vector_index = VectorStoreIndex.from_documents(documents)
# Create the query engine
query_engine = vector_index.as_query_engine()

# For the evaluation
# Set the LLMs
generator_llm = OpenAI(model=GENERATOR_LLM)
critic_llm = OpenAI(model=CRITIC_LLM)
evaluator_llm = OpenAI(model=EVALUATOR_LLM)
# Set the embeddings
embeddings = OpenAIEmbedding(model=EVALUATOR_EMBEDDING)

# Create the testset generator
generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

# Generate the testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    test_size=10,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

# Define the evaluation metrics
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

# convert to HF dataset
ds = testset.to_dataset()

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds,
    llm=OpenAI(model=EVALUATOR_LLM),
    embeddings=OpenAIEmbedding(model=EVALUATOR_EMBEDDING),
)

# Convert the result to pandas
result_df = result.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

max retries exceeded for SimpleEvolution(generator_llm=LlamaIndexLLMWrapper(run_config=RunConfig(timeout=180, max_retries=15, max_wait=90, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x30c303cd0>, nodes=[Node(metadata={'page_label': '1', 'file_name': 'Trump.pdf', 'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/eval/data/Trump/Trump.pdf', 'file_type': 'application/pdf', 'file_size': 86251, 'creation_date': '2024-10-15', 'last_modified_date': '2024-10-15'}, page_content='FormerPresidentDonaldTrumpwasshotandinjuredinanassassinationattempton\nSaturdaynightthatalsokilledaspectatorandcriticallyinjuredtwoothers.Sniperskilled\ntheshooter,a20-year-oldman,afterhefiredeightroundsattherallyinButler,\nPennsylvania.\nTheformerpresidentcouldbeseentouchinghisearastheshootingunfoldedbeforehe\nwasshieldedbySecretServiceandwhiskedoffstage—withbloo

Running Query Engine:   0%|          | 0/8 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs


In [8]:
result_df

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,Who is Thomas Matthew Crooks and what was his ...,[Former President Donald Trump was shot and in...,Thomas Matthew Crooks was a 20-year-old man fr...,Thomas Matthew Crooks was the shooter who atte...,1.000000,0.857161,1.0,1.000000
1,"Who described Crooks as super smart, approacha...","[Koehler, had a somewhat different view, telli...","Mark Sigafoos described Crooks as super smart,...","Mark Sigafoos, who graduated with Crooks and h...",1.000000,0.000000,1.0,1.000000
2,What was the identity of the victim who was fa...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,The victim who was fatally shot at the Trump r...,The victim who was fatally shot at the Trump r...,0.000000,0.943064,0.0,0.000000
3,How did the Secret Service respond to the shoo...,"[Convention in Milwaukee, with his right ear b...",The Secret Service responded to the shooting i...,The Secret Service responded to the shooting i...,0.833333,0.803050,1.0,1.000000
4,How far away was the shooter from the stage wh...,"[Convention in Milwaukee, with his right ear b...",The shooter was approximately 410 feet away fr...,The shooter was approximately 410 feet away fr...,1.000000,1.000000,1.0,1.000000
5,How did President Biden emphasize unity in res...,[has been getting briefed by top officials and...,President Biden emphasized unity by stating th...,President Biden emphasized unity in response t...,0.750000,0.885907,1.0,1.000000
6,What did President Biden say about recent viol...,[has been getting briefed by top officials and...,President Biden stated that there is no place ...,President Biden condemned the recent violence ...,1.000000,0.941360,1.0,0.666667
7,What was President Biden's response to the Pen...,[has been getting briefed by top officials and...,President Biden responded to the Pennsylvania ...,President Biden condemned the violence at the ...,0.888889,0.943581,1.0,1.000000


### Evaluator LLM using gpt-4o/ RAG using gpt-3.5-turbo

In [3]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.integrations.llama_index import evaluate

from dotenv import load_dotenv

load_dotenv()
# Get the API key from the environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# For different LLM models
RAG_LLM = "gpt-3.5-turbo"
GENERATOR_LLM = "gpt-4o"
CRITIC_LLM = "gpt-4o"
EVALUATOR_LLM = "gpt-4o"

# For different temperatures
TEMPERATURE = 0.1

# For different embeddings
RAG_EMBEDDING = "text-embedding-ada-002"
EVALUATOR_EMBEDDING = "text-embedding-ada-002"

# For different chunk
CHUNK_SIZE = 512
CHUNK_OVERLAP = 20


# Set the LLM, Embedding, and Text Splitter
Settings.llm = OpenAI(model=RAG_LLM, temperature=TEMPERATURE)
Settings.embed_model = OpenAIEmbedding(model=RAG_EMBEDDING)
Settings.text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)


# Load the documents
documents = SimpleDirectoryReader("data/Trump/").load_data()
# Create the vector index
vector_index = VectorStoreIndex.from_documents(documents)
# Create the query engine
query_engine = vector_index.as_query_engine()

# For the evaluation
# Set the LLMs
generator_llm = OpenAI(model=GENERATOR_LLM)
critic_llm = OpenAI(model=CRITIC_LLM)
evaluator_llm = OpenAI(model=EVALUATOR_LLM)
# Set the embeddings
embeddings = OpenAIEmbedding(model=EVALUATOR_EMBEDDING)

# Create the testset generator
generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

# Generate the testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    test_size=10,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

# Define the evaluation metrics
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

# convert to HF dataset
ds = testset.to_dataset()

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds,
    llm=OpenAI(model=EVALUATOR_LLM),
    embeddings=OpenAIEmbedding(model=EVALUATOR_EMBEDDING),
)

# Convert the result to pandas
result_df = result.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

max retries exceeded for MultiContextEvolution(generator_llm=LlamaIndexLLMWrapper(run_config=RunConfig(timeout=180, max_retries=15, max_wait=90, max_workers=16, exception_types=(<class 'Exception'>,), log_tenacity=False, seed=42)), docstore=InMemoryDocumentStore(splitter=<langchain_text_splitters.base.TokenTextSplitter object at 0x311773e50>, nodes=[Node(metadata={'page_label': '1', 'file_name': 'Trump.pdf', 'file_path': '/Users/chris/Desktop/7980/CS7980_Capstone_RBCMuseum/eval/data/Trump/Trump.pdf', 'file_type': 'application/pdf', 'file_size': 86251, 'creation_date': '2024-10-15', 'last_modified_date': '2024-10-15'}, page_content='FormerPresidentDonaldTrumpwasshotandinjuredinanassassinationattempton\nSaturdaynightthatalsokilledaspectatorandcriticallyinjuredtwoothers.Sniperskilled\ntheshooter,a20-year-oldman,afterhefiredeightroundsattherallyinButler,\nPennsylvania.\nTheformerpresidentcouldbeseentouchinghisearastheshootingunfoldedbeforehe\nwasshieldedbySecretServiceandwhiskedoffstage—wi

Running Query Engine:   0%|          | 0/8 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/32 [00:00<?, ?it/s]

n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs


In [4]:
result_df

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,"What happened at the Butler, Pennsylvania rall...","[Convention in Milwaukee, with his right ear b...","At the Butler, Pennsylvania rally involving fo...",Former President Donald Trump was shot and inj...,1.000000,1.000000,0.0,0.000000
1,What was the location of the Trump rally where...,"[Convention in Milwaukee, with his right ear b...",The Trump rally where the shooting took place ...,The Trump rally where the shooting took place ...,1.000000,0.975200,1.0,1.000000
2,What action did President Biden take in respon...,[has been getting briefed by top officials and...,President Biden ordered an independent review ...,President Biden first responded to the shootin...,1.000000,0.961421,1.0,1.000000
3,How did the Secret Service respond to the shoo...,"[Convention in Milwaukee, with his right ear b...",The Secret Service responded to the shooting i...,The Secret Service responded to the shooting i...,0.800000,1.000000,1.0,1.000000
4,What agencies are assisting the FBI in the inv...,[notontheradaroflawenforcement.\nTheFBIsaysinv...,The Secret Service and state and local agencie...,The FBI is leading the investigation with assi...,1.000000,0.966969,1.0,1.000000
5,Who was the hero fatally shot at the Trump ral...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,The hero fatally shot at the Trump rally while...,The hero fatally shot at the Trump rally while...,0.666667,1.000000,0.0,0.000000
6,What is the FBI investigating regarding the at...,[inthegunman'svehicle.Thegunmanalsohadwithhima...,The FBI is investigating the motive behind the...,The FBI is investigating the attempt on Donald...,1.000000,0.927596,1.0,0.666667
7,What led to the assassination attempt on Forme...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,The assassination attempt on Former President ...,The answer to given question is not present in...,1.000000,0.992235,0.0,1.000000


### Evaluator LLM gpt-4o/ RAG LLM gpt-4o

In [5]:
import os
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)
from ragas.integrations.llama_index import evaluate

from dotenv import load_dotenv

load_dotenv()
# Get the API key from the environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# For different LLM models
RAG_LLM = "gpt-4o"
GENERATOR_LLM = "gpt-4o"
CRITIC_LLM = "gpt-4o"
EVALUATOR_LLM = "gpt-4o"

# For different temperatures
TEMPERATURE = 0.1

# For different embeddings
RAG_EMBEDDING = "text-embedding-ada-002"
EVALUATOR_EMBEDDING = "text-embedding-ada-002"

# For different chunk
CHUNK_SIZE = 512
CHUNK_OVERLAP = 20


# Set the LLM, Embedding, and Text Splitter
Settings.llm = OpenAI(model=RAG_LLM, temperature=TEMPERATURE)
Settings.embed_model = OpenAIEmbedding(model=RAG_EMBEDDING)
Settings.text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)


# Load the documents
documents = SimpleDirectoryReader("data/Trump/").load_data()
# Create the vector index
vector_index = VectorStoreIndex.from_documents(documents)
# Create the query engine
query_engine = vector_index.as_query_engine()

# For the evaluation
# Set the LLMs
generator_llm = OpenAI(model=GENERATOR_LLM)
critic_llm = OpenAI(model=CRITIC_LLM)
evaluator_llm = OpenAI(model=EVALUATOR_LLM)
# Set the embeddings
embeddings = OpenAIEmbedding(model=EVALUATOR_EMBEDDING)

# Create the testset generator
generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

# Generate the testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    test_size=10,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

# Define the evaluation metrics
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]

# convert to HF dataset
ds = testset.to_dataset()

result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds,
    llm=OpenAI(model=EVALUATOR_LLM),
    embeddings=OpenAIEmbedding(model=EVALUATOR_EMBEDDING),
)

# Convert the result to pandas
result_df = result.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Running Query Engine:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs
n values greater than 1 not support for LlamaIndex LLMs


In [6]:
result_df

,question,contexts,answer,ground_truth,faithfulness,answer_relevancy,context_precision,context_recall
0,What was the position of the gunman during the...,[ofButlerCounty.\nThegunmanwasontheroofofabuil...,The gunman was on the roof of a building outsi...,The gunman was positioned on the roof of a bui...,1.000000,0.967744,1.0,1.0
1,How did the Secret Service respond to the shoo...,"[Convention in Milwaukee, with his right ear b...",The Secret Service responded to the shooting i...,The Secret Service responded to the shooting i...,1.000000,1.000000,1.0,1.0
2,Who was identified as the shooter in the assas...,[FormerPresidentDonaldTrumpwasshotandinjuredin...,The shooter in the assassination attempt on Do...,The shooter in the assassination attempt on Do...,1.000000,1.000000,1.0,1.0
3,How did President Biden emphasize the importan...,[has been getting briefed by top officials and...,President Biden emphasized the importance of u...,President Biden emphasized the importance of u...,0.666667,0.952612,1.0,1.0
4,How did the Secret Service respond to the shoo...,"[Convention in Milwaukee, with his right ear b...",The Secret Service responded to the shooting i...,The Secret Service responded to the shooting i...,1.000000,1.000000,1.0,1.0
5,Who protected his family from gunfire at the T...,"[Convention in Milwaukee, with his right ear b...","Corey Comperatore, a 50-year-old man, protecte...",Corey Comperatore protected his family from gu...,1.000000,1.000000,1.0,1.0
6,What political affiliation did Crooks demonstr...,[camo/huntingoutfitsinclass.HesaidCrookswouldo...,Crooks demonstrated a level of political compl...,Crooks demonstrated a political affiliation by...,1.000000,0.936507,1.0,1.0
7,What was Crooks' political affiliation given h...,[camo/huntingoutfitsinclass.HesaidCrookswouldo...,Crooks' political affiliation is not immediate...,The answer to given question is not present in...,1.000000,0.000000,1.0,1.0
8,What was Crooks' political affiliation in rela...,"[Koehler, had a somewhat different view, telli...",Crooks' political affiliation was not immediat...,The answer to given question is not present in...,0.750000,0.000000,0.0,1.0
9,What was found in the gunman's vehicle that ra...,[inthegunman'svehicle.Thegunmanalsohadwithhima...,Suspicious devices were found in the gunman's ...,Suspicious devices were found in the gunman's ...,1.000000,0.991028,1.0,1.0
